# Part 2: boost-histogram plans and Hist

## Boost-histogram numbafication plans

## Hist

Hist extends boost-histogram with:
* Named axes and labels
* UHI+: faster, easier to type indexing additions
* QuickConstruct: a system to reduce the typing when making histograms
* Fancy Jupyter reprs
* Other useful shortcuts, like density, pandas support, profiles from existing histograms, and more!

In [ ]:
# TODO: Add features since roughly 2.0.0 beta 1